In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357273 sha256=e3ae17041f3c3a2291dc63a25a66fae9857e6aacf6e88edd88a8f1d62e16e6f3
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [16]:
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

# Завантаження датасету
data = Dataset.load_builtin('ml-100k')
print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    print(row)

Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [17]:
# Розділення на тренувальну і тестову вибірки
trainset, testset = train_test_split(data, test_size=0.25)
# Алгоритм 1: SVD
algo_svd = SVD()
algo_svd.fit(trainset)
predictions_svd = algo_svd.test(testset)
# Алгоритм 2: KNN
algo_knn = KNNBasic()
algo_knn.fit(trainset)
predictions_knn = algo_knn.test(testset)

mae_svd = accuracy.mae(predictions_svd)
mae_knn = accuracy.mae(predictions_knn)

param_grid_svd = {'n_factors': [50, 100], 'reg_all': [0.02, 0.05]}
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=5)
gs_svd.fit(data)
param_grid_knn = {'k': [20, 40], 'min_k': [1, 5]}
gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['mae'], cv=5)
gs_knn.fit(data)

# Найкращі параметри
print("Best parameters for SVD:", gs_svd.best_params['mae'])
print("Best parameters for KNN:", gs_knn.best_params['mae'])

# Вибір найкращого алгоритму
if mae_svd < mae_knn:
    print("SVD має кращий результат")
    best_algo = algo_svd
else:
    print("KNN має кращий результат")
    best_algo = algo_knn

Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  0.7387
MAE:  0.7728
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similar

In [18]:
best_algo.fit(trainset)
# Виведення рекомендацій для конкретного користувача
user_id = str(196)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_algo.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 31
Топ-10 фільмів, рекомендованих для користувача:
Фільм 408 з прогнозованим рейтингом 4.83
Фільм 56 з прогнозованим рейтингом 4.70
Фільм 127 з прогнозованим рейтингом 4.68
Фільм 285 з прогнозованим рейтингом 4.64
Фільм 64 з прогнозованим рейтингом 4.62
Фільм 187 з прогнозованим рейтингом 4.57
Фільм 318 з прогнозованим рейтингом 4.56
Фільм 12 з прогнозованим рейтингом 4.55
Фільм 603 з прогнозованим рейтингом 4.49
Фільм 197 з прогнозованим рейтингом 4.44


**Висновок:** Для виконання завдання було завантажено датасет рецензій (ml-100k) за допомогою бібліотеки Surprise, що дозволило отримати дані про рейтинги користувачів для фільмів. Після виведення перших п'яти рядків, реалізував два алгоритми для побудови рекомендаційної системи: SVD та KNN. Для обох моделей застосував крос-валідацію з метою підбору оптимальних параметрів, використовуючи GridSearchCV. Після цього на основі середньої абсолютної помилки (MAE) обрав найкращу модель. За допомогою цієї моделі згенерував рекомендації для конкретного користувача.

**Завдання 2**

In [29]:
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import AlgoBase
from surprise import PredictionImpossible
import numpy as np

data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.25)
# Створення власного алгоритму для рекомендацій
class SimpleRecommender(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        self.global_mean = np.mean([rating for (_, _, rating) in trainset.all_ratings()])
        return self

    def estimate(self, u, i):
        if not self.trainset.knows_user(u) or not self.trainset.knows_item(i):
            raise PredictionImpossible('User or item unknown.')
        return self.global_mean

algo_simple = SimpleRecommender()
algo_simple.fit(trainset)
# Оцінка моделі
predictions = algo_simple.test(testset)
accuracy.mae(predictions)
# Отримання рекомендації для користувача
user_id = 202
movies_to_recommend = [i for i in range(1, 1683) if not trainset.knows_item(i)]
recommendations = [(i, algo_simple.predict(user_id, i).est) for i in movies_to_recommend]
recommendations.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача ", user_id)
for movie_id, rating in recommendations[:10]:
    print(f"Фільм {movie_id} з прогнозованим рейтингом {rating:.2f}")


MAE:  0.9429
Топ-10 фільмів, рекомендованих для користувача  202
Фільм 1635 з прогнозованим рейтингом 3.53
Фільм 1636 з прогнозованим рейтингом 3.53
Фільм 1637 з прогнозованим рейтингом 3.53
Фільм 1638 з прогнозованим рейтингом 3.53
Фільм 1639 з прогнозованим рейтингом 3.53
Фільм 1640 з прогнозованим рейтингом 3.53
Фільм 1641 з прогнозованим рейтингом 3.53
Фільм 1642 з прогнозованим рейтингом 3.53
Фільм 1643 з прогнозованим рейтингом 3.53
Фільм 1644 з прогнозованим рейтингом 3.53


**Висновок:** Під час виконання другого завдання була побудована власна рекомендаційна система. Для цього було розроблено алгоритм, який на основі середнього значення рейтингів фільмів робить прогнози для користувачів. Після тренування системи оцінив її ефективність, використовуючи метрику середньої абсолютної помилки (MAE). Після цього я згенерував рекомендації для конкретного користувача, запропонувавши фільми з найвищими прогнозованими оцінками, які користувач ще не переглядав.